In [185]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, copy, os, shutil
from tqdm.notebook import tqdm
from IPython.display import clear_output

In [156]:
# get our column names for our dataframe + create the dataframe
columns = ["PS", "PW", "MM", "theta", "t-color", "t-in", "t-out", 
           "IM", "NC", "K", "p-shuf", "max-iters", "eps", "a", "g", "seed", "num_episodes", 
           "mean_total_reward", "mean_steps_river", "mean_path_length", "mean_wallclock_time",
           "mean20_total_reward", "mean20_steps_river", "mean20_path_length", "mean20_wallclock_time"]
df = pd.DataFrame(columns=columns, data=None)

# what are our settings
fnames = [f for f in sorted(os.listdir("results")) if "PS" in f]

# go thru all of our settings
for fname in tqdm(fnames):
    
    # load in each seed
    trial_names = sorted([f for f in os.listdir(f"results/{fname}") if ".csv" in f])
    for trial_name in trial_names:
        
        # start the row with our settings
        row = [fname.split("PS")[1].split("=")[1].split("_")[0]]
        for col in columns[1:15]:
            if ("_" + col + "=") in fname:
                row.append(fname.split("_" + col + "=")[1].split("_")[0])
            else:
                row.append(None)
        
        # get our seed + add it to our columns
        seed = int(trial_name.split(".csv")[0].split("=")[1])
        row.append(seed)
        
        # load in our files + record our metrics of interest
        logs = pd.read_csv(f"results/{fname}/{trial_name}")
        
        # number of successfully completed episodes
        num_episodes = len(logs.index)
        
        # total means over all episodes
        mean_total_reward = logs.total_reward.mean()
        mean_steps_river = logs.steps_river.mean()
        mean_path_length = logs.path_length.mean()
        mean_wallclock_time = logs.wall_clock_time.mean()
        
        # let's look at last 20 episodes too, if applicable
        if num_episodes >= 20:
            mean20_total_reward = logs.total_reward.values[-20:].mean()
            mean20_steps_river = logs.steps_river.values[-20:].mean()
            mean20_path_length = logs.path_length.values[-20:].mean()
            mean20_wallclock_time = logs.wall_clock_time.values[-20:].mean()
        else:
            mean20_total_reward = logs.total_reward.mean()
            mean20_steps_river = logs.steps_river.mean()
            mean20_path_length = logs.path_length.mean()
            mean20_wallclock_time = logs.wall_clock_time.mean()
        
        # add to our row
        row += [num_episodes, mean_total_reward, mean_steps_river, mean_path_length, mean_wallclock_time, 
                mean20_total_reward, mean20_steps_river, mean20_path_length, mean20_wallclock_time]
        
        # add to our dataframe
        df.loc[len(df.index)] = row
        
# casting to various types as necessary
df.PS = df.PS.astype(float)
df.PW = df.PW.astype(float)
df.theta = df.theta.astype(float)
df["t-color"] = df["t-color"].astype(float)
df["t-in"] = df["t-in"].astype(float)
df["t-out"] = df["t-out"].astype(float)
df.NC = df.NC.astype(float)
df.K = df.K.astype(float)
df["p-shuf"] = df["p-shuf"].astype(float)
df["max-iters"] = df["max-iters"].astype(float)
df.eps = df.eps.astype(float)
df.a = df.a.astype(float)
df.g = df.g.astype(float)

# save our results
df.to_csv("aggregate_logs.csv", index=False)

  0%|          | 0/6480 [00:00<?, ?it/s]